# MNSYNLPM - Match Exporter and Analyzer for League of Legends

In [1]:
import json
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path

# Charger tous les fichiers JSON du dossier data
data_dir = Path("data")
all_matches = []

for json_file in data_dir.glob("*.json"):
    with open(json_file, "r", encoding="utf-8") as f:
        match_data = json.load(f)
        all_matches.append(match_data)

print(f"Nombre de matchs chargés : {len(all_matches)}")

Nombre de matchs chargés : 6


In [13]:

from src.constant import VIRGULE

# Extraire TOUTES les statistiques des joueurs Virgule avec calcul par minute
virgule_players_data = []

for match in all_matches:
    game_duration_sec = match.get("game", {}).get("gameDuration", 0)
    game_duration_min = game_duration_sec / 60 if game_duration_sec > 0 else 1
    
    # Pour chaque joueur Virgule, extraire ses stats complètes
    for player_name in VIRGULE:
        if player_name in match:
            player_data = match[player_name]
            champion_data = player_data.get("Champion", {})
            damage_data = player_data.get("Damage", {})
            kda_data = player_data.get("KDA", {})
            first_data = player_data.get("First", {})
            spell_data = player_data.get("Spell", {})
            ping_data = player_data.get("Ping", {})
            
            # Calculer les totaux de pings
            total_pings = sum([
                ping_data.get("onMyWay", 0),
                ping_data.get("danger", 0),
                ping_data.get("getBack", 0),
                ping_data.get("enemyMissing", 0),
                ping_data.get("assistMe", 0),
                ping_data.get("retreat", 0),
                ping_data.get("enemyVision", 0),
                ping_data.get("hold", 0),
                ping_data.get("needVision", 0),
                ping_data.get("push", 0),
                ping_data.get("visionCleared", 0),
                ping_data.get("allIn", 0),
                ping_data.get("basic", 0),
            ])
            
            # Récupérer les stats brutes
            kills = kda_data.get("kills", 0)
            deaths = kda_data.get("deaths", 0)
            assists = kda_data.get("assists", 0)
            gold = champion_data.get("gold", 0)
            cs = champion_data.get("creeps", 0)
            damage_dealt = damage_data.get("totalDamageDealt", 0)
            damage_taken = damage_data.get("totalDamageTaken", 0)
            
            # Créer le dictionnaire de données du joueur
            player_row = {
                "player_name": player_name,
                "champion_name": champion_data.get("championName"),
                "position": player_data.get("Position"),
                "level": champion_data.get("championLevel"),
                "game_duration_min": round(game_duration_min, 2),
                
                # Stats brutes
                "kills": kills,
                "deaths": deaths,
                "assists": assists,
                "gold": gold,
                "cs": cs,
                "damage_dealt": damage_dealt,
                "damage_taken": damage_taken,
                "turret_takedowns": champion_data.get("turretTakedowns", 0),
                "warding_placed": champion_data.get("wardsPlaced", 0),
                "warding_killed": champion_data.get("wardsKilled", 0),
                "vision_wards": champion_data.get("visionWardsPlaced", 0),
                "vision_score": champion_data.get("visionScore", 0),
                "time_spent_dead": champion_data.get("totalTimeSpentDead", 0),
                
                # Damage breakdown
                "physical_damage": damage_data.get("physicalDamageDealt", 0),
                "magic_damage": damage_data.get("magicDamageDealt", 0),
                "true_damage": damage_data.get("trueDamageDealt", 0),
                "damage_to_buildings": damage_data.get("damageDealtToBuildings", 0),
                "damage_to_epic_monsters": damage_data.get("damageDealtToEpicMonsters", 0),
                
                # KDA details
                "double_kills": kda_data.get("doubleKills", 0),
                "triple_kills": kda_data.get("tripleKills", 0),
                "quad_kills": kda_data.get("quadraKills", 0),
                "penta_kills": kda_data.get("pentaKills", 0),
                "largest_multi_kill": kda_data.get("largestMultiKill", 0),
                
                # First bloods
                "first_blood": first_data.get("firstBlood", False),
                "first_tower": first_data.get("firstTower", False),
                
                # Spells
                "q_casts": spell_data.get("Q_spell", 0),
                "w_casts": spell_data.get("W_spell", 0),
                "e_casts": spell_data.get("E_spell", 0),
                "r_casts": spell_data.get("R_spell", 0),
                "summoner1_casts": spell_data.get("summoner1Casts", 0),
                "summoner2_casts": spell_data.get("summoner2Casts", 0),
                
                # Pings
                "total_pings": total_pings,
                "ping_on_my_way": ping_data.get("onMyWay", 0),
                "ping_danger": ping_data.get("danger", 0),
                
                # Stats par minute
                "kills_per_min": round(kills / game_duration_min, 2),
                "deaths_per_min": round(deaths / game_duration_min, 2),
                "assists_per_min": round(assists / game_duration_min, 2),
                "gold_per_min": round(gold / game_duration_min, 2),
                "cs_per_min": round(cs / game_duration_min, 2),
                "damage_per_min": round(damage_dealt / game_duration_min, 2),
                "damage_taken_per_min": round(damage_taken / game_duration_min, 2),
                "warding_per_min": round(champion_data.get("wardsPlaced", 0) / game_duration_min, 2),
            }
            
            virgule_players_data.append(player_row)

df_virgule = pd.DataFrame(virgule_players_data)
print(f"Nombre de joueurs Virgule enregistrés : {len(df_virgule)}")
print(f"\nNombre de colonnes : {len(df_virgule.columns)}")
print(f"\nColonnes disponibles :")
print(df_virgule.columns.tolist())
print("\nPremières lignes du dataframe (stats brutes + par minute) :")
display(df_virgule.head(10))

Nombre de joueurs Virgule enregistrés : 30

Nombre de colonnes : 47

Colonnes disponibles :
['player_name', 'champion_name', 'position', 'level', 'game_duration_min', 'kills', 'deaths', 'assists', 'gold', 'cs', 'damage_dealt', 'damage_taken', 'turret_takedowns', 'warding_placed', 'warding_killed', 'vision_wards', 'vision_score', 'time_spent_dead', 'physical_damage', 'magic_damage', 'true_damage', 'damage_to_buildings', 'damage_to_epic_monsters', 'double_kills', 'triple_kills', 'quad_kills', 'penta_kills', 'largest_multi_kill', 'first_blood', 'first_tower', 'q_casts', 'w_casts', 'e_casts', 'r_casts', 'summoner1_casts', 'summoner2_casts', 'total_pings', 'ping_on_my_way', 'ping_danger', 'kills_per_min', 'deaths_per_min', 'assists_per_min', 'gold_per_min', 'cs_per_min', 'damage_per_min', 'damage_taken_per_min', 'warding_per_min']

Premières lignes du dataframe (stats brutes + par minute) :


,player_name,champion_name,position,level,game_duration_min,kills,deaths,assists,gold,cs,...,ping_on_my_way,ping_danger,kills_per_min,deaths_per_min,assists_per_min,gold_per_min,cs_per_min,damage_per_min,damage_taken_per_min,warding_per_min
0,matise,Swain,TOP,19,38.00,3,10,8,12807,255,...,1,0,0.08,0.26,0.21,337.03,6.71,914.63,1693.89,0.29
1,Fear of women,Vi,JUNGLE,17,38.00,10,10,8,15925,202,...,44,0,0.26,0.26,0.21,419.08,5.32,462.34,1127.08,0.18
2,Shao Mao,Lux,MIDDLE,17,38.00,4,7,12,13578,280,...,3,0,0.11,0.18,0.32,357.32,7.37,883.08,414.16,0.34
3,Ersees,Varus,BOTTOM,17,38.00,5,8,11,18206,310,...,7,0,0.13,0.21,0.29,479.11,8.16,815.92,643.03,0.11
4,nathboy,Poppy,SUPPORT,15,38.00,4,6,15,10947,43,...,10,0,0.11,0.16,0.39,288.08,1.13,316.45,753.29,1.92
5,matise,Jax,TOP,16,32.72,1,7,1,10030,219,...,0,0,0.03,0.21,0.03,306.57,6.69,0.00,0.00,0.00
6,Fear of women,Viego,JUNGLE,15,32.72,1,6,3,11324,179,...,0,0,0.03,0.18,0.09,346.12,5.47,0.00,0.00,0.00
7,Sabri,Ahri,MIDDLE,15,32.72,0,4,2,9767,256,...,0,0,0.00,0.12,0.06,298.53,7.82,0.00,0.00,0.00
8,Ersees,Ezreal,BOTTOM,15,32.72,3,4,2,13455,285,...,0,0,0.09,0.12,0.06,411.26,8.71,0.00,0.00,0.00
9,nathboy,Bard,SUPPORT,13,32.72,0,5,6,7511,32,...,0,0,0.00,0.15,0.18,229.58,0.98,0.00,0.00,0.00


## Statistiques globales des joueurs Virgule

In [5]:

# Moyenne par joueur
print("Moyenne des stats par joueur Virgule :")
stats_by_player = df_virgule.groupby("player_name").agg({
    "kills": "mean",
    "deaths": "mean",
    "assists": "mean",
    "gold": "mean",
    "cs": "mean",
    "level": "mean"
}).round(2)
print(stats_by_player)

Moyenne des stats par joueur Virgule :
               kills  deaths  assists      gold      cs  level
player_name                                                   
Ersees          5.33    4.50     9.33  17056.83  292.50  16.17
Fear of women   5.33    5.00     7.83  13820.83  207.83  16.50
Sabri           2.33    3.33     3.00  13542.33  297.67  17.00
Shao Mao        5.67    3.33    10.67  12963.67  244.33  16.67
matise          6.50    5.17     4.17  14141.67  250.50  18.00
nathboy         1.17    4.33    14.50   9855.67   34.67  14.17


In [6]:

# KDA ratio
df_virgule["kda_ratio"] = (df_virgule["kills"] + df_virgule["assists"]) / (df_virgule["deaths"] + 1)

print("\nKDA (Kills/Deaths/Assists) par joueur :")
kda_stats = df_virgule.groupby("player_name").agg({
    "kills": "sum",
    "deaths": "sum",
    "assists": "sum",
    "kda_ratio": "mean"
}).round(2)
print(kda_stats)


KDA (Kills/Deaths/Assists) par joueur :
               kills  deaths  assists  kda_ratio
player_name                                     
Ersees            32      27       56       3.74
Fear of women     32      30       47       3.23
Sabri              7      10        9       1.47
Shao Mao          17      10       32       7.67
matise            39      31       25       3.80
nathboy            7      26       87       3.37


## Graphiques - Performance des joueurs

In [7]:

# Graphique KDA par joueur
fig_kda = px.bar(
    kda_stats.reset_index(),
    x="player_name",
    y=["kills", "deaths", "assists"],
    title="KDA par joueur Virgule (Total)",
    barmode="group",
    labels={"player_name": "Joueur", "value": "Nombre"}
)
fig_kda.update_layout(height=500, width=900, template="plotly_white")
fig_kda.show()

In [8]:

# Graphique Gold et CS par joueur
avg_stats = df_virgule.groupby("player_name").agg({
    "gold": "mean",
    "cs": "mean"
}).reset_index()

fig_gold_cs = go.Figure()
fig_gold_cs.add_trace(go.Bar(
    x=avg_stats["player_name"],
    y=avg_stats["gold"],
    name="Gold moyen",
    marker_color="#FFD700"
))
fig_gold_cs.add_trace(go.Bar(
    x=avg_stats["player_name"],
    y=avg_stats["cs"],
    name="CS moyen",
    marker_color="#4169E1"
))
fig_gold_cs.update_layout(
    title="Gold et CS moyens par joueur Virgule",
    barmode="group",
    height=500,
    width=900,
    template="plotly_white",
    yaxis_title="Valeur"
)
fig_gold_cs.show()

In [9]:

# Graphique KDA ratio
kda_ratio_stats = df_virgule.groupby("player_name")["kda_ratio"].mean().reset_index()
kda_ratio_stats = kda_ratio_stats.sort_values("kda_ratio", ascending=False)

fig_ratio = px.bar(
    kda_ratio_stats,
    x="player_name",
    y="kda_ratio",
    title="KDA Ratio par joueur Virgule (+ élevé = meilleur)",
    color="kda_ratio",
    color_continuous_scale="Viridis",
    labels={"player_name": "Joueur", "kda_ratio": "KDA Ratio"}
)
fig_ratio.update_layout(height=500, width=900, template="plotly_white")
fig_ratio.show()

## Statistiques par champion

In [10]:

# Stats par champion
champion_stats = df_virgule.groupby("champion_name").agg({
    "kills": ["mean", "sum"],
    "deaths": ["mean", "sum"],
    "assists": ["mean", "sum"],
    "gold": "mean",
    "cs": "mean",
    "level": "mean"
}).round(2)

print("Statistiques par champion (Champions joués par Virgule) :")
display(champion_stats)

Statistiques par champion (Champions joués par Virgule) :


kills     deaths     assists         gold     cs level
               mean sum   mean sum    mean sum     mean   mean  mean
champion_name                                                       
Ahri            5.0  10    2.0   4     4.5   9  11646.5  242.0  15.5
Alistar         0.0   0    3.0   3    13.0  13  11008.0   32.0  16.0
Aphelios        8.0   8    1.0   1    13.0  13  16699.0  259.0  16.0
Bard            0.0   0    5.0   5     6.0   6   7511.0   32.0  13.0
Braum           1.0   1    4.0   4    21.0  21   7928.0   26.0  12.0
DrMundo         3.0   3    5.0   5     4.0   4  15237.0  253.0  18.0
Ezreal          3.0   3    4.0   4     2.0   2  13455.0  285.0  15.0
Fiora           6.0   6    7.0   7     3.0   3  18177.0  334.0  20.0
Jax             1.0   1    7.0   7     1.0   1  10030.0  219.0  16.0
KSante          8.0   8    0.0   0     6.0   6  11135.0  185.0  16.0
Lux             4.0   4    7.0   7    12.0  12  13578.0  280.0  17.0
Milio           1.0   1    2.0   2    20.0  20   9194.0   35.0  14.0
MonkeyKing      5.0   5    4.0   4     9.0   9  16061.0  265.0  18.0
Poppy           4.0   4    6.0   6    15.0  15  10947.0   43.0  15.0
Rakan           1.0   1    6.0   6    12.0  12  12546.0   40.0  15.0
RekSai          8.0   8    4.0   4     4.0   4  18329.0  309.0  20.0
Ryze            5.0   5    2.0   2     4.0   4  15954.0  307.0  18.0
Sett           13.0  13    3.0   3     3.0   3  14372.0  201.0  17.0
Swain           3.0   3   10.0  10     8.0   8  12807.0  255.0  19.0
Syndra          2.0   2    4.0   4     3.0   3  14906.0  330.0  18.0
Varus           4.5   9    5.5  11    10.5  21  18699.5  315.0  17.5
Vi             10.0  10   10.0  10     8.0   8  15925.0  202.0  17.0
Viego           1.0   1    6.0   6     3.0   3  11324.0  179.0  15.0
Viktor          3.0   3    3.0   3    13.0  13  11787.0  225.0  17.0
Volibear        7.0   7    4.0   4     9.0   9  12498.0  175.0  16.0
Xayah           6.0   6    7.0   7     4.0   4  22571.0  416.0  18.0
Yunara          6.0   6    4.0   4    16.0  16  12217.0  165.0  13.0
Zaahen          6.0   6    1.0   1    14.0  14  11880.0  173.0  15.0

## Comparaison entre les matches

In [11]:

# Moyenne des stats de l'équipe Virgule par match
team_stats = []
for match in all_matches:
    virgule_data = match.get("virgule", {})
    virgule_names = virgule_data.get("picks", [])
    
    match_players = [match[name] for name in virgule_names if name in match]
    
    if match_players:
        total_kills = sum([p.get("KDA", {}).get("kills", 0) for p in match_players])
        total_deaths = sum([p.get("KDA", {}).get("deaths", 0) for p in match_players])
        total_assists = sum([p.get("KDA", {}).get("assists", 0) for p in match_players])
        avg_gold = sum([p.get("Champion", {}).get("gold", 0) for p in match_players]) / len(match_players)
        avg_cs = sum([p.get("Champion", {}).get("creeps", 0) for p in match_players]) / len(match_players)
        
        team_stats.append({
            "match": list(match.keys())[0] if match else "unknown",
            "total_kills": total_kills,
            "total_deaths": total_deaths,
            "total_assists": total_assists,
            "avg_gold": avg_gold,
            "avg_cs": avg_cs,
            "win": virgule_data.get("win", False)
        })

df_team_stats = pd.DataFrame(team_stats)
print("Statistiques d'équipe Virgule par match :")
display(df_team_stats)

Statistiques d'équipe Virgule par match :


""


In [12]:

# Graphique performance globale par match
fig_match = go.Figure()

fig_match.add_trace(go.Bar(
    x=df_team_stats["match"],
    y=df_team_stats["total_kills"],
    name="Total Kills",
    marker_color="#FF6B6B"
))

fig_match.add_trace(go.Bar(
    x=df_team_stats["match"],
    y=df_team_stats["total_assists"],
    name="Total Assists",
    marker_color="#4ECDC4"
))

fig_match.update_layout(
    title="Kills et Assists totaux de Virgule par match",
    barmode="group",
    height=500,
    width=1000,
    template="plotly_white",
    yaxis_title="Nombre"
)
fig_match.show()

KeyError: 'match'